In [ ]:
#Need to Validate that Linear Regression is Correclty including Definition.
# Updated Raw Data which is validated in Git.

In [146]:
import pandas as pd
import numpy as np
import datetime

import sys
sys.path.append("/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions")

google_notes_csv = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSQF2lNc4WPeTRQ_VzWPkqSZp4RODFkbap8AqmolWp5bKoMaslP2oRVVG21x2POu_JcbF1tGRcBgodu/pub?output=csv'
google_definition_csv = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQq1-3cTas8DCWBa2NKYhVFXpl8kLaFDohg0zMfNTAU_Fiw6aIFLWfA5zRem4eSaGPa7UiQvkz05loW/pub?output=csv'

notes = pd.read_csv(google_notes_csv).fillna('')
definitions = pd.read_csv(google_definition_csv).fillna('')

ml_def = definitions.copy()
ml_notes = notes[notes['Category'].isin(['Machine Learning','Visualizations'])]

from df_processing import notes_df_to_outline_html,final_dataset_for_markdown
d = final_dataset_for_markdown(ml_notes,ml_def)
d

,Category,Categorization,Word,Notes
0,Machine Learning,Definition,Machine Learning,Using a process to enable computers to iterati...
4,Machine Learning,Data Preperation,Goal,NaN
5,Machine Learning,Data Preperation,Approach,NaN
6,Machine Learning,Data Preperation,Important to Remeber,NaN
7,Machine Learning,Data Preperation,Lesson Learnt,NaN
...,...,...,...,...
72,Machine Learning,Tuning,Important to Remember,NaN
73,Machine Learning,Tuning,Lesson Learnt,NaN
74,Machine Learning,Tuning,Algorithm,NaN
75,Machine Learning,"Bias, Fairness and Ethics",,NaN


In [152]:
def final_dataset_for_markdown(notes=None,
                               definitions=None,
                               export_location='/Users/derekdewald/Documents/Python/Github_Repo/Streamlit/DataDictionary/'):
    
    '''
    
    Function which helps to combined Notes and Definitions into a Single Combined Representation which can ultimately be used as a Learning Reference Tools.

    How this function Works:
    It takes the two sheets and attemps to Consolidate them together to make Final Dataset, generated as d_learning_notes.csv.

    Step 1. Add a new Row Record, which is the Definition for Category and record where Definition is the Category and Definition.
            As a Model of Notes, I will not include the Categorization Definition, as it is redudant, the entire purpose is that everything
            should have a definition, this is the method to add it. Need to ensure Notes Process follows this.
    Step 2. Add Definitions Matching Category and Categorization. Add Category Level Definitions (ex. Machine Learning/ Activation Function)
    Step 3. Add Definitions Matching Category and Word. Add Record Level Observations (ex. Machine Learning/ Supervised Learning)
    Step 4. Add Effective Parameter Listing, where I can Expand Upon Ideas where Lists or Options are important, without overly explicitly idenfying.
            Match notes.Categorization= definitions.category and notes.word=definitions.categorizaztion
            Linear Regression: 
                Notes: Categorization: Model, Word: Algorithm
                Definitions: Category: Model, Categorization: Algorithm
    
    Parameters:
        notes(df): DataFrame of D Notes as stored in: https://docs.google.com/spreadsheets/d/e/2PACX-1vSQF2lNc4WPeTRQ_VzWPkqSZp4RODFkbap8AqmolWp5bKoMaslP2oRVVG21x2POu_JcbF1tGRcBgodu/pub?output=csv
        definitions(df): DataFrame of D Definitions as stored in: https://docs.google.com/spreadsheets/d/e/2PACX-1vQq1-3cTas8DCWBa2NKYhVFXpl8kLaFDohg0zMfNTAU_Fiw6aIFLWfA5zRem4eSaGPa7UiQvkz05loW/pub?output=csv

    Returns:
        DF

    date_created:20-Dec-25
    date_last_modified: 20-Dec-25
    classification:TBD
    sub_classification:TBD
    usage:
        final_dataset_for_markdown()


    ##############

    Has been tested for a Single Value - Machine Learning. Need to Validate once extending.

    ##############

    
    '''

    from data_d_dicts import links

    try:
        len(notes)
    except:
        notes = pd.read_csv(links['google_notes_csv'])

    try:
        len(definitions)
    except:
        definitions = pd.read_csv(links['google_definition_csv'])

    merge_def = definitions[['Word','Definition','Category']]

    # Step 1
    # Create a DF of Unique Records in Category on Notes Sheet, with Categorization of Definition.
    temp_df = notes.drop_duplicates("Category")[['Category']]
    temp_df['Categorization'] = 'Definition'
    temp_df['Word'] = temp_df['Category']

    # Merging to include Definition of Category as a Word
    temp_df1 = temp_df.merge(merge_def.rename(columns={'Definition':"Notes"}),on=['Word','Category'],how='left')   
    temp_df1['Notes'] = temp_df1['Notes'].fillna('Not Defined')

    # Step 1B Need to Add Combindation of Category and Categorization = Definition from D Notes.
    temp_df1a =  temp_df.drop('Word',axis=1).merge(definitions[['Category','Categorization','Word','Definition']].rename(columns={'Definition':"Notes"}),on=['Category','Categorization'],how='left')
    
    # Drop Instances Where there are no definitions
    temp_df1a = temp_df1a[temp_df1a['Word'].notnull()].copy()

    # Add the Step 1 and Step 1A
    temp_df1 = pd.concat([temp_df1,temp_df1a])
    temp_df1['RANK'] = 0 

    # Step 2
    # Add Definitions to all Unique Combinations of Category and Categorization on Notes Sheet
    temp_df2 = notes.drop_duplicates(['Category','Categorization']).drop(['Word','Notes'],axis=1).copy()
    temp_df2 = temp_df2.merge(definitions[['Word','Definition','Category']].rename(columns={'Word':'Categorization','Definition':"Notes"}),on=['Category','Categorization'],how='left')
    temp_df2['Notes'] = temp_df2['Notes'].fillna('Not Defined')
    temp_df2['Word'] = temp_df2['Categorization'].copy()
    temp_df2['RANK'] = 0 

    # Step 3
    # Add Definitions for all Combination of Word and Category
    temp_df3 = notes.merge(merge_def,on=['Word','Category'],how='left')
    temp_df3['Definition'] = temp_df3['Definition'].fillna('Not Defined')
    temp_df3['RANK'] = 1

    # Step 4
    # Merge In Listed Parameters 
    temp_df4 =  notes[['Category','Categorization','Word']].drop_duplicates(['Categorization','Word'])

    # Generate Information From Definition. Must remove Note column from current dataframe to accept for purposes of merging.
    list_ = definitions.drop('Notes',axis=1).rename(columns={'Word':'Notes','Categorization':'Word','Category':'Categorization'})[['Categorization','Word','Notes','Definition']]
    list_["Notes"] = list_["Notes"].astype(str) + ": " + list_["Definition"].astype(str)
    list_.drop('Definition',axis=1,inplace=True)
    
    # Merge Into Notes.
    temp_df4 =  temp_df4.merge(list_,on=['Categorization','Word'],how='inner')

    # Create Final DataFrame
    final_df = pd.concat([temp_df1,temp_df2,temp_df3,temp_df4])

    # Base Column is Notes, Definitions Merged in to Help Articulate and combine, thus if Notes Available, it will take precendent
    final_df['Notes'] = np.where(final_df['Notes']=="",final_df['Definition'],final_df['Notes'])
    final_df.drop('Definition',axis=1,inplace=True)
    
    # Calculuate Order DF. Using Order as defined in Notes Template.
    order_df =  notes[['Category','Categorization']].drop_duplicates().reset_index(drop=True).reset_index().rename(columns={'index':'ORDER'})
    final_df = final_df.merge(order_df,on=['Category','Categorization'],how='left')
    final_df['ORDER'] = final_df['ORDER'].fillna(0) 
    
    # Sort    
    final_df =  final_df.sort_values(['Category','ORDER','RANK']).drop(['ORDER','RANK'],axis=1)

    # Remove Blank Place Holders that were satisfied by Merging Parameter Listing (Optimization, Optimizers as Example)
    final_df['count'] = final_df.groupby(['Category','Categorization','Word']).transform('size')
    final_df = final_df[~((final_df['count']>1)&(final_df['Notes'].isnull()))].copy()
    final_df = final_df.drop('count',axis=1).reset_index(drop=True)

    if export_location:
        final_df.to_csv(f"{export_location}d_learning_notes.csv",index=False)
    
    return final_df

In [153]:
d = final_dataset_for_markdown(ml_notes,ml_def)
d

,Category,Categorization,Word,Notes
0,Machine Learning,Definition,Machine Learning,Using a process to enable computers to iterati...
1,Machine Learning,Definition,Activation Function,Used during learning and prediction in a neura...
2,Machine Learning,Definition,Area Under the Curve,AUC (Area Under the Curve) measures a model’s ...
3,Machine Learning,Definition,Autoregressive,Autoregressive models generate outputs one ste...
4,Machine Learning,Definition,Bias,Bias refers to the systematic error in a model...
...,...,...,...,...
310,Machine Learning,Tuning,Algorithm,Not Defined
311,Machine Learning,"Bias, Fairness and Ethics","Bias, Fairness and Ethics",Not Defined
312,Machine Learning,"Bias, Fairness and Ethics",,Not Defined
313,Machine Learning,Model Interpretability / Explainability,Model Interpretability / Explainability,Not Defined


In [99]:
definitions[['Category','Categorization','Word','Definition']]

,Category,Categorization,Word,Definition
0,Machine Learning,Definition,Supervised Learning,Supervised learning trains a model using label...
1,Machine Learning,Definition,Unsupervised Learning,Unsupervised learning works with unlabeled dat...
2,Machine Learning,Definition,Semisupervised Learning,Semi-supervised learning combines a small amou...
3,Machine Learning,Definition,Reinforcement Learning,Reinforcement learning trains an agent to make...
4,Machine Learning,Definition,Machine Learning,Using a process to enable computers to iterati...
...,...,...,...,...
254,Machine Learning,Definition,Variance,Variance refers to errors caused by models tha...
255,Machine Learning,Definition,Bias - Variance Trade Off,The bias-variance trade-off is a key concept i...
256,Machine Learning,Definition,Area Under the Curve,AUC (Area Under the Curve) measures a model’s ...
257,Machine Learning,Definition,Autoregressive,Autoregressive models generate outputs one ste...


In [96]:
ml_def.head()

,Category,Categorization,Word,Definition,Notes,Link,Image,Markdown Equation,Dataset Size,Learning Type,Algorithm Class
0,Machine Learning,Definition,Supervised Learning,Supervised learning trains a model using label...,,,,,,,
1,Machine Learning,Definition,Unsupervised Learning,Unsupervised learning works with unlabeled dat...,,,,,,,
2,Machine Learning,Definition,Semisupervised Learning,Semi-supervised learning combines a small amou...,,,,,,,
3,Machine Learning,Definition,Reinforcement Learning,Reinforcement learning trains an agent to make...,,,,,,,
4,Machine Learning,Definition,Machine Learning,Using a process to enable computers to iterati...,,,,,,,


           Category Categorization              Word  \
0  Machine Learning     Definition  Machine Learning   
1    Visualizations     Definition    Visualizations   

                                               Notes  
0  Using a process to enable computers to iterati...  
1                                                NaN  
                      Word                                         Definition  \
0      Supervised Learning  Supervised learning trains a model using label...   
1    Unsupervised Learning  Unsupervised learning works with unlabeled dat...   
2  Semisupervised Learning  Semi-supervised learning combines a small amou...   
3   Reinforcement Learning  Reinforcement learning trains an agent to make...   
4         Machine Learning  Using a process to enable computers to iterati...   

           Category  
0  Machine Learning  
1  Machine Learning  
2  Machine Learning  
3  Machine Learning  
4  Machine Learning  


KeyError: 'Categorization'

In [88]:
d[d['Word'].str.contains('var')]

,Category,Categorization,Word,Notes


### How do Definitions Get Added to final_dataset_for_markdown
### Examples
#### Variance
#### Bias - Variance Trade Off
#### Area Under the Curve
